In [1]:
import requests
from datetime import datetime
import pandas as pd
import re 
import csv
import time
import mysql.connector
import pymysql, os, json
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy import create_engine



In [2]:
now = datetime.today()

In [3]:
list_province_in_thai = []
list_district_in_thai = []
list_subdistrict_in_thai = []
label = pd.read_excel('../code/geocodelabel.xlsx', index_col=None)
for i in range(len(label)):
    list_province_in_thai.append(label['name_th'][i])
    list_district_in_thai.append(label['district_name_th'][i])
    list_subdistrict_in_thai.append(label['subdistrict_name_th'][i])

# clear dup
list_province_in_thai = list(dict.fromkeys(list_province_in_thai))
list_district_in_thai = list(dict.fromkeys(list_district_in_thai))
list_subdistrict_in_thai = list(dict.fromkeys(list_subdistrict_in_thai))


In [4]:
def longdoAddress(lat,lng):
    url = "https://api.longdo.com/map/services/address?"
    params = {'lat': lat , 'lon' : lng , 'key' : '1917e02b4ab264ba547fecdd6e82b4aa'}
    response =requests.get(url, params=params)
    payload = response.json()
    #print("------------------------------------------")
    #print(payload)
    #print("------------------------------------------")
    province = removeWrongkeyword(payload['province'])
    district = removeWrongkeyword(payload['district'])
    subdistrict = removeWrongkeyword(payload['subdistrict'])
    try:
        postcode = payload['postcode']
        
    except :
        postcode = ""
    
    time.sleep(1)
    
    return province,district,subdistrict,postcode

In [5]:
def extracteachDetail(word):
    payload_word = word
    if(re.findall(r"\จังหวัด",payload_word)):
        payload_word = re.sub("จังหวัด","",payload_word)
    elif(re.findall(r"\อำเภอ",payload_word)):
        payload_word = re.sub("อำเภอ","",payload_word)
    elif(re.findall(r"\ตำบล",payload_word)):
        payload_word = re.sub("ตำบล","",payload_word)
    elif(re.findall(r"\เขต",payload_word)):
        payload_word = re.sub("เขต","",payload_word)
    elif(re.findall(r"\แขวง",payload_word)):
        payload_word = re.sub("แขวง","",payload_word)
        
    return payload_word

In [6]:
def removeWrongkeyword(word):
    payload_word = word
    if(re.findall(r"\จ+[.]",word)):
        payload_word = re.sub(r"\จ+[.]","จังหวัด",word)
    elif(re.findall(r"\อ+[.]",word)):
        payload_word = re.sub(r"\อ+[.]","อำเภอ",word)
    elif(re.findall(r"\ต+[.]",word)):
        payload_word = re.sub(r"\ต+[.]","ตำบล",word)
    return payload_word
        

In [7]:
def getdataFromAPI():
    url = "https://www.cjexpress.co.th/branch/all"
    response = requests.get(url)
    payload = response.json()
    with open(f'../trash/CJ{str(now.date())}.json', 'w') as f:
        json.dump(payload, f)

In [8]:
getdataFromAPI()

In [9]:
t = open(f'../trash/CJ{str(now.date())}.json')
payload = json.load(t)

In [10]:
payload

[{'code': 790,
  'province': 12,
  'branch': 'SCวังตาผิน',
  'address': 'เลขที่ 69/92 หมู่ที่ 4 ตำบลปลวกแดง อำเภอปลวกแดง จังหวัดระยอง 21140',
  'tel': '061-3847031',
  'lat': 12.979881,
  'lng': 101.194,
  'card': 0,
  'emoney': 1},
 {'code': 379,
  'province': 11,
  'branch': 'SP บ้านสวน',
  'address': 'เลขที่ 99/376 หมู่ที่ 7 ตำบลบ้านสวน อำเภอเมืองชลบุรี จังหวัดชลบุรี 20000',
  'tel': '064-7957735',
  'lat': 13.372707,
  'lng': 101.009758,
  'card': 0,
  'emoney': 0},
 {'code': 675,
  'province': 1,
  'branch': 'The rich park พหลโยธิน 59',
  'address': 'เลขที่ 251/2-6 ถนนพหลโยธิน แขวงอนุสาวรีย์  เขตบางเขน  กรุงเทพมหานคร 10220',
  'tel': '095-8033410',
  'lat': 13.882466,
  'lng': 100.600479,
  'card': 0,
  'emoney': 1},
 {'code': 309,
  'province': 12,
  'branch': 'กรอกยายชา',
  'address': 'เลขที่ 18/8  ถนนกรอกยายชา ตำบลเนินพระ  อำเภอเมืองระยอง  จังหวัดระยอง  21000',
  'tel': '081-3784872',
  'lat': 12.679917,
  'lng': 101.198418,
  'card': 0,
  'emoney': 1},
 {'code': 915,
  'provin

In [11]:
def getAddressDetail(address,lat,lng):
    str_address = address.replace("  "," ")
    #print(str_address)

    #remove newline 
    if("\n" in str_address) :
        str_address = re.sub('\r','',str_address)
        str_address = re.sub('\n',' ',str_address)
    #print(str_address)
    # fix missing keyword
    temp_fix = str_address.split()
    
    if(len(temp_fix[-3]) == 2):
        temp_fix.pop(-3)
    
        #check if don't have postcode or missing keyword
    if(re.findall("[ก-ฮ]", temp_fix[-1])):
        #print(str_address)
        #print(temp_fix[-1])
            #find missing province 
        if(temp_fix[-1] not in list_province_in_thai):
            # 0 = จังหวัด , 1 = อำเภอ , 2 = ตำบล
            pro,_,_,postcode = longdoAddress(lat,lng)
            temp_fix.append(pro)
            temp_fix.append(postcode)
            #print(temp_fix)
            time.sleep(5)
    
    else :
        if(re.findall("[ก-ฮ]", temp_fix[-2])):
            #print("yest")
            # check and fix province contain จ.
            if(re.findall(r"\จ+[.]", temp_fix[-2])):
                temp_fix[-2] = removeWrongkeyword(temp_fix[-2])
            if(re.findall(r"\อ+[.]", temp_fix[-3])):
                temp_fix[-3] = removeWrongkeyword(temp_fix[-3])
            if(re.findall(r"\ต+[.]", temp_fix[-4])):
                temp_fix[-4] = removeWrongkeyword(temp_fix[-4])
    
    #fix prefix 
    if(not re.findall("จังหวัด", temp_fix[-2])):
        temp_fix[-2] = "จังหวัด"+ temp_fix[-2]
    if(not re.findall("อำเภอ", temp_fix[-3])):
        temp_fix[-3] = "อำเภอ"+ temp_fix[-3]
    if(not re.findall("ตำบล", temp_fix[-4])):
        temp_fix[-4] = "ตำบล"+ temp_fix[-4]
    fix_address = ""
    for j in temp_fix :
        fix_address = fix_address + j + " "
    
    #print(fix_address)
    
    # get each detail
    province = temp_fix[-2].replace("จังหวัด","")
    district = temp_fix[-3].replace("อำเภอ","")
    subdistrict = temp_fix[-4].replace("ตำบล","")
    
    is_find = False
    
    if(province == "กรุงเทพฯ"):
        province = province.replace("กรุงเทพฯ","กรุงเทพมหานคร")
    if(province not in list_province_in_thai):
        print(str_address)
        is_find = True
        province,district,subdistrict,postcode = longdoAddress(lat,lng)
        postcode = extracteachDetail(postcode)
        province = extracteachDetail(province)
        district = extracteachDetail(district)
        subdistrict = extracteachDetail(subdistrict)
    if(district not in list_district_in_thai):
        if(not is_find):
            print(str_address)
            is_find = True
            province,district,subdistrict,postcode = longdoAddress(lat,lng)
            postcode = extracteachDetail(postcode)
            province = extracteachDetail(province)
            district = extracteachDetail(district)
            subdistrict = extracteachDetail(subdistrict)
    if(subdistrict not in subdistrict):
        if(not is_find):
            print(str_address)
            is_find = True
            province,district,subdistrict,postcode = longdoAddress(lat,lng)
            postcode = extracteachDetail(postcode)
            province = extracteachDetail(province)
            district = extracteachDetail(district)
            subdistrict = extracteachDetail(subdistrict)
    
    
    '''
    print("province: "+ province)
    print("district: "+district)
    print("subdistrict: "+subdistrict)
    print("-------------------------------------------")
    '''
    return fix_address,province,district,subdistrict

In [12]:
'''
{'code': 371,
  'province': 3,
  'branch': 'ชลลดา',
  'address': 'เลขที่ 24/24  หมู่ที่ 1  ตำบลพิมลราช  อำเภอบางบัวทอง  จังหวัดนนทบุรี  11110',
  'tel': '088-9231639',
  'lat': 13.911167,
  'lng': 100.429253,
  'card': 0,
  'emoney': 1},
 {'code': 268,
  'province': 56,
  'branch': 'ช่องสะเดา',
  'address': 'เลขที่ 234  หมู่ที่ 1  ตำบลช่องสะเดา  อำเภอเมืองกาญจนบุรี  จังหวัดกาญจนบุรี  71000',
  'tel': '064-1841804',
  'lat': 14.223445,
  'lng': 99.238869,
  'card': 1,
  'emoney': 1},
 '''

"\n{'code': 371,\n  'province': 3,\n  'branch': 'ชลลดา',\n  'address': 'เลขที่ 24/24  หมู่ที่ 1  ตำบลพิมลราช  อำเภอบางบัวทอง  จังหวัดนนทบุรี  11110',\n  'tel': '088-9231639',\n  'lat': 13.911167,\n  'lng': 100.429253,\n  'card': 0,\n  'emoney': 1},\n {'code': 268,\n  'province': 56,\n  'branch': 'ช่องสะเดา',\n  'address': 'เลขที่ 234  หมู่ที่ 1  ตำบลช่องสะเดา  อำเภอเมืองกาญจนบุรี  จังหวัดกาญจนบุรี  71000',\n  'tel': '064-1841804',\n  'lat': 14.223445,\n  'lng': 99.238869,\n  'card': 1,\n  'emoney': 1},\n "

In [13]:
def getStorename(id):
    store_type = ""
    if(id == 0):
        store_type = "CJ Supermarket"
    elif(id == 1):
        store_type = "CJ Express"
    return store_type

In [14]:
def getStoretype(id):
    store_type = ""
    if(id == 0):
        store_type = "Supermarket"
    elif(id == 1):
        store_type = "Convenience Stores"
    return store_type

In [15]:
list_CJ_supermarket = []
list_CJ_express = []
for i in range(len(payload)):
    name = payload[i]['branch']
    store_id = payload[i]['code']
    address = payload[i]['address']
    lat = payload[i]['lat']
    lng = payload[i]['lng']
    no_type = payload[i]['card']
    address,province,district,subdistrict = getAddressDetail(address,lat,lng)
    store_name = getStorename(no_type)
    store_type = getStoretype(no_type)
    dict_CJ = {"name" : name , "store_id" : store_id,
                        "name_store" : store_name , "store_type" : store_type,
                        "address" : address , "lat" : lat , "lng" : lng ,
                        "subdistrict" : subdistrict , "district" : district ,"province" : province 
                }
    if(no_type == 0) :
        list_CJ_supermarket.append(dict_CJ)
    elif(no_type == 1):
        list_CJ_express.append(dict_CJ)
        
    print(i)


0
1
เลขที่ 251/2-6 ถนนพหลโยธิน แขวงอนุสาวรีย์ เขตบางเขน กรุงเทพมหานคร 10220
2
3
4
5
6
7
8
เลขที่ 433/1 ซอยกรุงธนบุรี 6 ถนนกรุงธนบุรี แขวงบางลำภูล่าง เขตคลองสาน กรุงเทพมหานคร 10600
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
เลขที่ 129 หมู่ที่ 8 ตำบลหนองสามวัง อำเภอหนองเสือจังหวัดปทุมธานี 12170
35
36
37
38
39
40
41
42
43
44
45
46
47
เลขที่ 40/27 หมู่ที่ 8 แขวงคู้ฝั่งเหนือ เขตหนองจอก กรุงเทพมหานคร 10530
48
เลขที่ 249 ถนนบางแวก แขวงบางแวก เขตภาษีเจริญ กรุงเทพมหานคร 10160
49
เลขที่ 54 ซอยจรัญสนิทวงศ์ 75 แขวงบางพลัด เขตบางพลัด กรุงเทพมหานคร 10700
50
51
เลขที่ 71/180 หมู่ที่ 1 แขวงจอมทอง เขตจอมทอง กรุงเทพมหานคร 10150
52
53
54
55
56
57
58
59
60
61
62
63
64
เลขที่ 57/1 ถนนชักพระ แขวงคลองชักพระ เขตตลิ่งชัน กรุงเทพมหานคร 10170
65
66
67
68
69
70
71
72
73
74
75
เลขที่ 242 ถนนสรณคมน์ แขวงสีกัน เขตดอนเมือง กรุงเทพมหานคร 10210
76
เลขที่ 169/98 หมู่ที่ 5 ตำบลบ้านฉาง อำแภอบ้านฉาง จังหวัดระยอง 21130
77
78
79
80
81
82
83
84
85
86
87
88
เลขที่ 132 ซอย ม.อนันต์สุขสันต์รุ่น 

In [16]:
with open(f"../trash/CJsupermarket{str(now.date())}.json"  , "w") as outfile:
            json.dump(list_CJ_supermarket, outfile)
with open(f"../trash/CJexpress{str(now.date())}.json"  , "w") as outfile:
            json.dump(list_CJ_express, outfile)

In [17]:
df_json =  pd.read_json(f"../trash/CJexpress{str(now.date())}.json")
df_json.to_excel(f"../excel/CJ/CJexpress/CJexpress{str(now.date())}.xlsx")
df_json.to_csv(f"../csv/CJ/CJexpress/CJexpress{str(now.date())}.csv")


In [18]:
df_json =  pd.read_json(f"../trash/CJsupermarket{str(now.date())}.json")
df_json.to_excel(f"../excel/CJ/CJsupercenter/CJsupermarket{str(now.date())}.xlsx")
df_json.to_csv(f"../csv/CJ/CJsupercenter/CJsupermarket{str(now.date())}.csv")

In [ ]:
#TODBCJ
ce = pd.read_csv(f'../csv/CJ/CJexpress/CJexpress{str(now.date())}.csv', header = 0)
cs = pd.read_csv(f'../csv/CJ/CJsupercenter/CJsupermarket{str(now.date())}.csv', header = 0)
engine = create_engine('mysql://root:@localhost:3306/cjexpress')
with engine.connect() as conn, conn.begin():
    ce.to_sql(f'CJexpress{str(now.date())}', conn, if_exists='append', index=False)
conn.close()
engine = create_engine('mysql://root:@localhost:3306/cjsupermarket')
with engine.connect() as conn, conn.begin():
    cs.to_sql(f'CJsupermarket{str(now.date())}', conn, if_exists='append', index=False)
conn.close()